In [ ]:
# import needed packages

#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import colormaps
import cartopy.crs as ccrs #for plotting on map
import cartopy as cart
from datetime import datetime, timedelta
from importlib import reload
import hexbin_functions as hexfunc
import h3
from shapely.geometry import shape, Point

hexfunc = reload(hexfunc)
# test version, it should be version 4
print(h3.__version__)

from analysis_functions_xr import trajectory_length, Haversine_list, calc_tidal_av
from analysis_functions import make_PDF, Haversine
from particle_characteristics_functions import factor_drag_white1991, factor_drag_Schiller1933, factor_drag_morrison2013, Re_particle

# plotstyle: 
plt.style.use('../python_style_Meike.mplstyle')
# markers
color_array = np.array(['c','orange','purple','green','navy'])
marker_array = np.array(['s','o','>','p'])
line_array = np.array(['-','--','-.',':'])
markerline_array = np.array(['-s','--o','-.>',':p'])

In [ ]:
# set needed constants
# set needed constants
Rearth = 6371 * 10**3 # in m,
deg2rad = np.pi / 180.
sec_in_hours= 3600
diameter = 0.2 #m
rho_water = 1027 # kg/m3 https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
dynamic_viscosity_water = 1.41 * 10**(-3) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water

In [ ]:
# filenames
# import simulation data
Replist = np.array([0,10,100,1000])
coriolislist = [True,False]
gradientlist = [True,False]
B = 0.68
tau = 2994.76 #2759.97
runtime =  timedelta(days=30)# timedelta(days=10)
land_handling = 'anti_beaching'
nparticles = 88347 # 52511
chunck_time = 100
loc = 'NWES'
coriolis = True

starttimes = np.array([datetime(2023, 9, 1, 0, 0, 0, 0)]) #,
                    #     datetime(2023, 10, 1, 0, 0, 0, 0), #)
                    #    datetime(2023, 11, 1, 0, 0, 0, 0) ,
                    #    datetime(2023, 12, 1, 0, 0, 0, 0) ,
                    #    datetime(2024, 1, 1, 0, 0, 0, 0),
                    #    datetime(2024, 2, 1, 0, 0, 0, 0)])

date_new = datetime(2023, 9, 1, 0, 0, 0, 0)

base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'

basefile_Rep_constant = (base_directory + '{particle_type}/{loc}_'
                 'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                 'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_'
                 '_Rep_{Rep:04d}_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')



basefile_tracer  = (base_directory + '{particle_type}/{loc}_'
                        'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                        'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_{land_handling}.zarr')

basefile_MR_Rep_drag = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')


particle_types = ['tracer','inertial_SM_drag_Rep','inertial_SM_Rep_constant'] # 
simtype = {'tracer':'tracer','inertial_SM_Rep_constant':'SM MR','inertial_Rep_constant':'full MR'}

basefiles = {'tracer':basefile_tracer,
             'inertial_SM_drag_Rep':basefile_MR_Rep_drag,
             'inertial_Rep_constant':basefile_Rep_constant,
             'inertial_SM_Rep_constant':basefile_Rep_constant
             }

for Rep in Replist:
    print(f'c({Rep}) = {factor_drag_white1991(Rep):.02f}')


In [ ]:
# read in data
data ={}
for pt in particle_types:
    data[pt]={}
    for gradient in gradientlist:
        data[pt][gradient]={}
        if(pt in  ['tracer','inertial_SM_drag_Rep']):
            data[pt][gradient][None]={}
        else:
            for Rep in Replist:
                data[pt][gradient][Rep]={}


for pt in particle_types:
    for gradient in gradientlist:
        for starttime in starttimes:
            print(starttime)
            endtime = starttime + runtime 
            date = f'{starttime.year:04d}/{starttime.month:02d}'
            if(pt == 'tracer'):
                print (pt)
                file = basefile_tracer.format(loc=loc,
                                            y_s=starttime.year,
                                            m_s=starttime.month,
                                            d_s=starttime.day,
                                            y_e=endtime.year,
                                            m_e=endtime.month,
                                            d_e=endtime.day,
                                            land_handling = land_handling, 
                                            coriolis = coriolis,
                                            particle_type = pt)
                
                ds = xr.open_dataset(file,
                                    engine='zarr',
                                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                                    drop_variables=['z'],
                                    decode_times=False) #,decode_cf=False)

                data[pt][gradient][None][date]= ds 
            elif(pt == 'inertial_SM_drag_Rep'):
                print (pt)
                file = basefiles[pt].format(loc=loc,
                                                y_s=starttime.year,
                                                m_s=starttime.month,
                                                d_s=starttime.day,
                                                y_e=endtime.year,
                                                m_e=endtime.month,
                                                d_e=endtime.day,
                                                B = int(B * 1000), 
                                                tau = int(tau ),
                                                land_handling = land_handling, 
                                                coriolis = coriolis,
                                                particle_type = pt,
                                                gradient = gradient,)
                ds = xr.open_dataset(file,
                    engine='zarr',
                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                    drop_variables=['z'],
                    decode_times=False) #,decode_cf=False)

                data[pt][gradient][None][date]= ds 
            else:
                for Rep in Replist:
                    file = basefiles[pt].format(loc=loc,
                                                y_s=starttime.year,
                                                m_s=starttime.month,
                                                d_s=starttime.day,
                                                y_e=endtime.year,
                                                m_e=endtime.month,
                                                d_e=endtime.day,
                                                B = int(B * 1000), 
                                                tau = int(tau ),
                                                land_handling = land_handling, 
                                                coriolis = coriolis,
                                                particle_type = pt,
                                                Rep = Rep,
                                                gradient = gradient)
                    ds = xr.open_dataset(file,
                                        engine='zarr',
                                        chunks={'trajectory':nparticles, 'obs':chunck_time},
                                        drop_variables=['B','tau','z'],
                                        decode_times=False) #,decode_cf=False)

                    data[pt][gradient][Rep][date]= ds 



In [ ]:
Blues = colormaps['Blues']



# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.3, 0.7, 2))



fig,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.coastlines()
idstart = 6008 # np.random.randint(0,nparticles)#22#3748 #568#25680#47584#98
idstep = 1
idend = idstart+1 * idstep

ax.add_feature(cart.feature.LAND, facecolor='lightgrey')


# plot selection of trajectories
Blues = colormaps['Blues']
pt = particle_types[1]
Rep = 0
for gradient, color in zip(gradientlist,bluelist):
        year=2023
        month=9
        date = f'{year:04d}/{month:02d}'
        legend= []
        coriolis = True
        
        ax.coastlines()
        idstart = 900#000#25#6008#25#6008 # np.random.randint(0,nparticles)#22#3748 #568#25680#47584#987
        idstep = 1
        idend = idstart+1 * idstep

        ax.add_feature(cart.feature.LAND, facecolor='lightgrey')

        # plot tracer particle
        ax.plot(data['tracer'][coriolis][None][date].lon[idstart,0::1].T,
                data['tracer'][coriolis][None][date].lat[idstart,0::1].T,
                '-',
                color='k',zorder=20);
        legend.append('tracer')

        # ax.legend(particle_types,loc=(1,0.2))
        pt = 'inertial_SM_drag_Rep'
        for coriolis, color in zip(coriolislist,bluelist):
                ax.plot(data[pt][coriolis][None][date].lon[idstart:idend:idstep,0::1].T,
                        data[pt][coriolis][None][date].lat[idstart:idend:idstep,0::1].T,
                        '--',
                        color=color);

                
        ax.legend(['tracer','gradient on', 'gradient off'],loc = 2,fontsize=18,handlelength = 1.5) #, loc=(1,0.2)
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                linewidth=0, color='gray', alpha=0.5, linestyle='--')
        gl.top_labels = False
        gl.right_labels = False
        gl.xlabel_style = {'size': 18}
        gl.ylabel_style =  {'size': 18}
        # ax.set_xlim(-14.6,-10.5)
        # ax.set_ylim(54.4,56.1)

In [ ]:
Blues = colormaps['Blues']



# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.3, 0.7, 2))



fig,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.coastlines()
idstart = 6008 # np.random.randint(0,nparticles)#22#3748 #568#25680#47584#98
idstep = 1
idend = idstart+1 * idstep

ax.add_feature(cart.feature.LAND, facecolor='lightgrey')


# plot selection of trajectories
Blues = colormaps['Blues']
pt = particle_types[1]
Rep = 0
for gradient, color in zip(gradientlist,bluelist):
        year=2023
        month=9
        date = f'{year:04d}/{month:02d}'
        legend= []
        coriolis = True
        
        ax.coastlines()
        idstart = 25# 900#25# 900#25# 900#000#25#6008#25#6008 # np.random.randint(0,nparticles)#22#3748 #568#25680#47584#987
        idstep = 1
        idend = idstart+1 * idstep

        ax.add_feature(cart.feature.LAND, facecolor='lightgrey')

        # plot tracer particle
        ax.plot(data['tracer'][coriolis][None][date].lon[idstart,0::1].T,
                data['tracer'][coriolis][None][date].lat[idstart,0::1].T,
                '-',
                color='k',zorder=-20);
        legend.append('tracer')

        # ax.legend(particle_types,loc=(1,0.2))
        pt = 'inertial_SM_Rep_constant'
        Rep =100
        for coriolis, color in zip(coriolislist,bluelist):
                ax.plot(data[pt][coriolis][Rep][date].lon[idstart:idend:idstep,0::1].T,
                        data[pt][coriolis][Rep][date].lat[idstart:idend:idstep,0::1].T,
                        '--',
                        color=color);

                
        ax.legend(['tracer','gradient on', 'gradient off'],loc = 2,fontsize=18,handlelength = 1.5) #, loc=(1,0.2)
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                linewidth=0, color='gray', alpha=0.5, linestyle='-')
        gl.top_labels = False
        gl.right_labels = False
        gl.xlabel_style = {'size': 18}
        gl.ylabel_style =  {'size': 18}
        ax.set_xlim(-14.6,-10.5)
        ax.set_ylim(54.4,56.1)

fig.tight_layout()
fig.savefig('../figures/poster/trajectory_ID25_gradient.pdf')

In [ ]:
# show final shapshot
startime = starttimes[0]
year = starttime.year
month = starttime.month
date = f'{year:04d}/{month:02d}'
pt = 'inertial_SM_Rep_constant'
Tmax=700
fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
gradient = True
ax.scatter(data['tracer'][gradient][None][date].lon[:,Tmax],data['tracer'][gradient][None][date].lat[:,Tmax],marker='o',s=2,alpha=0.1,color='k')
ax.coastlines()
ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 20}
gl.ylabel_style =  {'size': 20}

Rep = 0
for gradient, color in zip(gradientlist,color_array):
    fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
    ax.scatter(data[pt][gradient][Rep][date].lon[:,Tmax],data[pt][gradient][Rep][date].lat[:,Tmax],marker='o',s=2,alpha=0.1,color=color)
    ax.coastlines()
    ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                linewidth=0, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'size': 20}
    gl.ylabel_style =  {'size': 20}


